## I - Enoncé du problème

#### Question 1. Comment peut-on formuler ce problème en termes de graphes ?

Pour formuler le problème du voyageur de commerce en termes de graphes, on peut représenter les villes comme des sommets d'un graphe et les distances entre les villes comme des arêtes pondérées entre ces sommets.
Le poids de chaque arête correspond à la distance entre les deux villes qu'elle connecte.

#### Question 2. Quel est le nombre de trajets possibles pour une instance à n villes ?

Pour une instance à n villes, le nombre de trajets possibles est déterminé par le nombre de permutations des villes.
Ainsi, on a (n-1)!  trajets possibles. Cette réduction s'explique par le fait qu'il n'est pas nécessaire de prendre en compte les permutations qui sont simplement des rotations ou des réflexions du même trajet.
Par exemple, pour une instance à 4 villes, il y a 3! = 6 trajets possibles à considérer.

## II - Résolution à l’aide d’un algorithme génétique

#### Question 3. Décrivez ce que sont les individus dans le cas présent. Comment les représenter par un chromosome ?

Dans le cas présent, les individus sont représentés par les trajets possibles entre les villes.
Un individu peut être représenté par un chromosome, qui est une séquence de gènes.
Chaque gène représente une ville, et la séquence de gènes dans le chromosome représente l'ordre dans lequel le voyageur visite les villes.

Par exemple, si nous avons 4 villes nommées A, B, C et D, un chromosome possible pourrait être [A, B, C, D], ce qui signifie que le voyageur visite les villes dans l'ordre A -> B -> C -> D -> A (retour à la ville de départ).

### 2.1 Classe Ville

#### Question 4. Créez une classe Ville ; le constructeur doit permettre d’initialiser une ville à l’aide de son nom, et de ses coordonnées x et y.

#### Question 5. Ajoutez une méthode distance_vers(autre_ville) qui renvoie la distance entre la ville courante et une autre ville passée en paramètre.

#### Question 6. Implémentez la méthode __str__ pour que la fonction print appelée sur un objet Ville affiche le nom de la ville.

### 2.2 Méthode utilitaires

#### Question 7. Créez une fonction globale nommée generer_villes. Cette fonction prend un paramètre optionnel nb_villes (dont la valeur par défaut est 20) et renvoie une liste d’objets Ville aux coordonnées aléatoires (comprises entre 0 et 300) ; le nom de la ville est simplement un numéro.

#### Question 8. A l’aide du module python csv, écrivez une fonction globale nommée lire_csv, qui prend un paramètre le nom d’un fichier CSV (sur chaque ligne, on trouve simplement le nom de la ville et ses coordonnées, le tout étant séparé par des virgules). La fonction lit le fichier et renvoie la liste d’objets Ville correspondante.

In [ ]:
import math
import random


class Ville:

    def __init__(self, nom, x, y):
        """
        Constructeur de la classe Ville

        :param nom: le nom de la ville
        :param x: la coordonnée x de la ville
        :param y: la coordonnée y de la ville
        """
        self.nom = nom
        self.x = x
        self.y = y

    def __str__(self):
        """
        Méthode appelée lorsqu'on affiche un objet Ville
        :return: le nom de la ville et ses coordonnées
        """
        return self.nom

    def distance_vers(self, autre_ville):
        """
        Calcule la distance entre la ville courante et une autre ville passée en paramètre.

        :param autre_ville: l'autre ville
        :return: la distance entre les deux villes
        """
        dx = self.x - autre_ville.x
        dy = self.y - autre_ville.y
        return math.sqrt(dx * dx + dy * dy)

    def generer_villes(nb_villes=20):
        """
        Génère une liste de villes aléatoires.
        :return: la liste de villes générées.
        """
        return [Ville(str(i), random.randint(0, 300), random.randint(0, 300)) for i in range(nb_villes)]

    def lire_csv(fic_name):
        """
        Lit un fichier CSV et renvoie la liste des villes correspondantes.
        :return: la liste des villes lues.
        """
        villes = []
        with open(fic_name, 'r') as fichier:
            for ligne in fichier:
                nom, x, y = ligne.split(',')
                villes.append(Ville(nom, int(x), int(y)))
        return villes

In [ ]:
villes = Ville.generer_villes()

for ville in villes:
    # affiche la distance avec une ville aléatoire de la liste
    print("Distance entre", ville, "et", random.choice(villes), ":", ville.distance_vers(random.choice(villes)))

### 2.3 Classe Trajet

Une Ville correspond à un gène, et un trajet, c’est-à-dire une suite de n villes, correspond donc à un chromosome.

#### Question 9. Créez la classe Trajet : ses attributs sont une liste de Ville et la longueur du trajet correspondant. Le constructeur prend en paramètre optionnel une liste de villes :
> - en l’absence de liste fournie, on initialise simplement une liste vide
> - si une liste est fournie, on génère un trajet aléatoire (regardez le module random de Python) à partir de cette liste (attention à ne pas modifier la liste passée en paramètre !)

#### Question 10. Créez une méthode calc_longueur servant à mettre à jour l’attribut longueur à chaque modification du trajet (pensez à reutiliser la méthode distance_vers de la classe Ville !).

#### Question 11. Créez une méthode est_valide permettant de vérifier qu’une liste de villes correspond bien à un trajet valide (c’est-à-dire que chaque ville n’est présente qu’une et une seule fois).

#### Question 12. Ecrivez la méthode __str__ pour que la représentation textuelle d’un trajet soit simplement la liste de ses villes.


In [ ]:
import random


class Trajet:
    def __init__(self, villes=None):
        """
        Constructeur de la classe Trajet

        :param villes: une liste de villes
        """

        # Si aucune liste de villes n'est fournie, on initialise une liste vide
        if villes is None:
            self.villes = []

        else:
            # Si une liste de villes est fournie, on génère un trajet aléatoire à partir de cette liste
            self.villes = random.sample(villes, len(villes))

        # Calcule et met à jour la longueur du trajet
        self.longueur = self.calc_longueur()

    def calc_longueur(self):
        """
        Calcule la longueur du trajet.

        :return: La longueur du trajet
        """
        longueur = 0
        if len(self.villes) > 1:

            # Parcours les villes dans l'ordre et calcule la distance entre chaque paire de villes consécutives
            for i in range(len(self.villes) - 1):
                longueur += self.villes[i].distance_vers(self.villes[i + 1])

            # Ajoute la distance entre la dernière ville et la première ville pour compléter le cycle
            longueur += self.villes[-1].distance_vers(self.villes[0])
        return longueur

    def est_valide(self):
        """
        Vérifie que le trajet est valide.
        Chaque ville doit être présente une seule fois dans le trajet.
        :return: True si le trajet est valide, False sinon
        """
        return len(set(self.villes)) == len(self.villes)

    def __str__(self):
        """
        Méthode appelée lorsqu'on affiche un objet Trajet.
        :return: La représentation textuelle du trajet.
        """
        return " -> ".join(str(ville) for ville in self.villes)


In [ ]:
villes = Ville.generer_villes()

trajet = Trajet(villes)
print(trajet)
print(f"Longueur du trajet : {trajet.longueur:.2f}")
print(f"Trajet valide : {trajet.est_valide()}")

### 2.4 Classe Population

A présent que vous pouvez créer des individus / trajets, vous allez écrire le code permettant de gérer une population.

#### Question 13. Créez la classe Population ; une population est simplement une liste de Trajet (initialement vide) (ceci nous permet de créer des populations vides auxquelles nous rajouterons par la suite des individus).

#### Question 14. Ajoutez la méthode initialiser(taille, liste_villes) : cette méthode initialise une population de taille individus, générés aléatoirement à partir de la liste des villes fournies (il suffit de faire appel au code de Trajet déjà écrit !).

#### Question 15. Ajoutez la méthode ajouter(trajet) qui, comme son nom l’indique, ajoute le trajet fourni à la population courante.

#### Question 16. Ajoutez la méthode meilleur() qui retourne le meilleur individu, c’est-à-dire le trajet de plus petite longueur dans la population (indice : comment trier une liste en Python ?).

#### Question 17. Ecrivez la méthode __str__ pour que la représentation textuelle d’une population soit simplement la liste des trajets qu’elle contient.

In [ ]:
class Population:
    def __init__(self):
        """
        Constructeur de la classe Population
        """
        self.trajets = []

    def initialiser(self, taille, liste_villes):
        """
        Initialise une population de taille individus, générés aléatoirement à partir de la liste des villes fournies.

        :param taille: La taille de la population
        :param liste_villes: La liste des villes à partir de laquelle générer les trajets
        """
        self.trajets = [Trajet(liste_villes) for _ in range(taille)]

    def ajouter(self, trajet):
        """
        Ajoute un trajet à la population.
        :param trajet: Le trajet à ajouter
        """
        self.trajets.append(trajet)

    def meilleur(self):
        """
        Retourne le meilleur individu, c'est-à-dire le trajet de plus petite longueur dans la population.
        :return: Le meilleur individu
        """
        return min(self.trajets, key=lambda trajet: trajet.longueur)

    def __str__(self):
        """
        Méthode appelée lorsqu'on affiche un objet Population.
        :return: La représentation textuelle de la population.
        """
        return "\n".join(str(trajet) for trajet in self.trajets)

In [ ]:
villes = Ville.generer_villes()

population = Population()
population.initialiser(5, villes)

print("Population :", population, sep="\n")

meilleur_trajet = population.meilleur()
print("\nMeilleur trajet :", meilleur_trajet, sep="\n")
print(f"Longueur du meilleur trajet : {meilleur_trajet.longueur:.2f}")


In [ ]:
from tkinter import *


class PVC_Genetique_GUI(object):
    """
    Runs the application
    """

    def __init__(self, liste_villes):
        self.liste_villes = liste_villes
        self.generation = 0

        # Initiates a window object & sets its title
        self.window = Tk()
        self.window.wm_title("Génération 0")

        # initiates two canvases, one for current and one for best
        self.canvas_current = Canvas(self.window, height=300, width=300)
        self.canvas_best = Canvas(self.window, height=300, width=300)

        # Initiates two labels
        self.canvas_current_title = Label(self.window, text="Meilleur trajet de la génération courante :")
        self.canvas_best_title = Label(self.window, text="Meilleur trajet trouvé jusqu'ici :")

        # Initiates a status bar with a string
        self.stat_tk_txt = StringVar()
        self.status_label = Label(self.window, textvariable=self.stat_tk_txt, relief=SUNKEN, anchor=W)

        # creates dots for the cities on both of the canvases
        for city in liste_villes:
            self.canvas_current.create_oval(city.x - 2, city.y - 2, city.x + 2, city.y + 2, fill='blue')
            self.canvas_best.create_oval(city.x - 2, city.y - 2, city.x + 2, city.y + 2, fill='blue')

        # Packs all the widgets (physically creates them and places them in order)
        self.canvas_current_title.pack()
        self.canvas_current.pack()
        self.canvas_best_title.pack()
        self.canvas_best.pack()
        self.status_label.pack(side=BOTTOM, fill=X)

        # Runs the main window loop
        self.window.update()

    def afficher(self, meilleur, courant, pas=1, afficher_noms=False):
        self.generation += 1
        if self.generation == 1:
            self.initial = courant.longueur
            if afficher_noms:
                for v in self.liste_villes:
                    self.canvas_best.create_text(v.x - 2, v.y - 5, text=v.nom, fill="black", font=('Helvetica 8'))
                    self.canvas_current.create_text(v.x - 2, v.y - 5, text=v.nom, fill="black", font=('Helvetica 8'))

        self.window.wm_title("Génération {0}".format(self.generation))
        self.update_canvas(self.canvas_best, meilleur, 'green')
        if self.generation % pas == 0:
            self.update_canvas(self.canvas_current, courant, 'red')

        self.stat_tk_txt.set(
            'Trajet initial {0:.2f}    Meilleur trajet = {1:.2f}'.format(self.initial, meilleur.longueur))
        self.status_label.pack()
        self.status_label.update_idletasks()

    def update_canvas(self, the_canvas, trajet, color):
        # deletes all current items with tag 'path'
        the_canvas.delete('path')

        # loops through the route
        for i in range(len(trajet.villes)):
            # similar to i+1 but will loop around at the end
            next_i = i - len(trajet.villes) + 1

            # creates the line from city to city
            the_canvas.create_line(trajet.villes[i].x,
                                   trajet.villes[i].y,
                                   trajet.villes[next_i].x,
                                   trajet.villes[next_i].y,
                                   tags=("path"),
                                   fill=color)

            # Packs and updates the canvas
            the_canvas.pack()
            the_canvas.update_idletasks()

    def GA_loop(self, n_generations, pop_size, graph=False):
        return

## Implémentation de l’algorithme génétique - Classe PVC_Genetique

Une classe PVC_Genetique est fournie (elle ne contient pour l’instant qu’une méthode utilitaire clear_term). Il s’agit de la classe principale de votre programme : c’est par elle qu’on va initialiser et exécuter l’algorithme génétique.

#### Question 18. Créez le constructeur de cette classe : il prend un paramètre obligatoire (la liste des villes) et deux paramètres optionnels (la taille de la population, par défaut à 40, et le nombre de générations, par défaut à 100).

#### Question 19. Écrivez la méthode croiser(parent1, parent2). Elle doit renvoyer l’enfant issu du croisement.
> Il faut bien veiller à ce que l’enfant généré corresponde à un trajet valide (on ne peut pas juste croiser les listes comme dans l’exemple du cours, car on pourrait avoir la même ville présente plusieurs fois dans le trajet) ! Proposez une méthode permettant de vous en assurer.

#### Question 20. Écrivez la méthode muter(trajet). Elle doit renvoyer le trajet muté.
> Ici aussi, on ne peut pas simplement remplacer une ville par une autre ville aléatoire. Quelle méthode de mutation proposez-vous pour garantir des trajets valides ?.
> Pensez bien à recalculer la longueur du trajet ainsi modifié !.

#### Question 21. Écrivez la méthode selectionner(population), correspondant à une méthode de sélection de votre choix pour les individus qui devront se reproduire (selon ce que vous mettrez en place, elle peut renvoyer directement une liste d’individus, ou bien un seul individus, auquel cas il faudra rappeler plusieurs fois la méthode selectionner). Vous disposez à présent de toutes les briques pour faire évoluer une population !

#### Question 22. Écrivez la méthode évoluer(population) : cette méthode utilise les méthodes précédentes de la classe PVC_Genetique pour faire évoluer une population d’individus, et retourne la nouvelle population.

#### Question 23. Ajoutez au constructeur deux paramètres optionnels : elitisme = True et mut_proba = 0.3, et modifiez votre compte pour prendre en compte l’élitisme lors de la sélection, et la valeur du paramètre mut_proba avant d’effectuer une mutation.

#### Question 24. Enfin, la méthode principale de la classe PVC_Genetique est la méthode executer. C’est elle qui crée la population initiale puis la fait évoluer sur le nombre de générations spécifié lors de l’appel au constructeur. Elle se charge également de conserver la trace du meilleur trajet trouvé.
> Vous pouvez, pour chaque génération, afficher le meilleur individu de cette génération ainsi que le meilleur individu depuis le début. Pour ne pas “polluer” votre terminal, vous pouvez utiliser la fonction clear_term fournie.
> Une classe PVC_Genetique_GUI vous est fournie dans le fichier GeneticTSPGui.py ; cette classe contient
tout le code nécessaire pour une représentation “graphique” de vos trajets.

#### Question 25. Pour l’utiliser, vous devez :
> - ajouter un attribut de type PVC_Genetique_GUI dans la classe PVC_Genetique ; le constructeur a besoin de connaître la liste des villes pour les afficher ;
> - ajouter dans votre méthode executer un attribut optionnel afficher = True
> - quand l’attribut afficher vaut True, faire appel à la méthode affiche() du PVC_Genetique_GUI ; cette méthode prend en paramètre le meilleur trajet global, ainsi que le meilleur trajet de la génération courante, et un paramètre booléen optionnel afficher_noms qui permet d’afficher les noms des villes sur les cartes.
> A la fin de la méthode exécuter, ajouter la ligne votre_objet_GUI.window.mainloop()


In [ ]:
import os

class PVC_Genetique:

    def clear_term(self):
        """
        Efface le terminal
        """
        os.system('cls' if os.name == 'nt' else 'clear')


    def __init__(self, liste_villes, taille_population=40, nb_generations=100):
        self.liste_villes = liste_villes
        self.taille_population = taille_population
        self.nb_generations = nb_generations

    def croiser(self, parent1, parent2):
        """
        Retourne l'enfant issu du croisement de deux parents

        :param parent1: Un Trajet 1
        :param parent2: Un Trajet 2
        :return: Un Trajet valide
        """
        child_cities = parent1.villes.copy()
        for v2 in parent2.villes:
            if v2 not in child_cities:
                child_cities.append(v2)
        return Trajet(child_cities)

    def muter(self, trajet):
        """
        Retourne un trajet muté

        :param trajet: Un Trajet
        """
        # sélectionne 2 villes aléatoires
        index1, index2 = random.sample(range(len(trajet.villes)), 2)
        # échange les villes
        trajet.villes[index1], trajet.villes[index2] = trajet.villes[index2], trajet.villes[index1]
        # recalcule la longueur du trajet
        trajet.longueur = trajet.calc_longueur()

    def selectionner(self, population):
        """
        Retourne un individu de la population

        :param population: Une liste de Trajet
        """
        # on trie la population par longueur de trajet
        population.trajets.sort(key=lambda x: x.longueur)
        # on retourne une liste des 20% meilleurs
        return population.trajets[:int(self.taille_population * 0.2)]

    def evoluer(self, population):
        """
        Retourne une nouvelle population

        :param population: Une liste de Trajet
        """
        nouvelle_population = Population()

        # on garde le meilleur trajet de la population précédente
        traj = self.selectionner(population)

        # on ajoute le meilleur trajet de la population précédente à la nouvelle population
        nouvelle_population.trajets.append(traj)






parent1 = Trajet(Ville.generer_villes(5))
parent2 = Trajet(Ville.generer_villes(10))
algorithme = PVC_Genetique([])
enfant = algorithme.croiser(parent1, parent2)
print("Parent 1 :", parent1)
print("Parent 2 :", parent2)
print("Enfant   :", enfant)

trajet = Trajet(Ville.generer_villes(5))
print("Trajet avant mutation :", trajet)
algorithme = PVC_Genetique([])
algorithme.muter(trajet)
print("Trajet après mutation :", trajet)

# selectionner
population = Population()
population.initialiser(5, Ville.generer_villes(5))
algorithme = PVC_Genetique([])
elite = algorithme.selectionner(population)
print("Population initiale :\n", population)
print("Elite :\n20% des meilleurs trajets :")
for trajet in elite:
    print(trajet)








In [ ]:
class PVC_Genetique:
    def clear_term(self):
        """
        Efface le terminal
        """
        os.system('cls' if os.name == 'nt' else 'clear')

    def __init__(self, liste_villes, taille_population=40, nb_generations=100, elitisme=True, mut_proba=0.3):
        self.liste_villes = liste_villes
        self.taille_population = taille_population
        self.nb_generations = nb_generations
        self.elitisme = elitisme
        self.mut_proba = mut_proba
        self.gui = PVC_Genetique_GUI(liste_villes)

    def croiser(self, parent1, parent2):
        taille = len(parent1.villes)
        start, end = sorted(random.sample(range(taille), 2))

        child_villes = parent1.villes[start:end]

        for ville in parent2.villes:
            if ville not in child_villes:
                child_villes.append(ville)

        return Trajet(child_villes)

    def muter(self, trajet):
        index1, index2 = random.sample(range(len(trajet.villes)), 2)
        trajet.villes[index1], trajet.villes[index2] = trajet.villes[index2], trajet.villes[index1]
        trajet.longueur = trajet.calc_longueur()

    def selectionner(self, population):
        population.trajets.sort(key=lambda x: x.longueur)
        return population.trajets[:int(self.taille_population * 0.2)]

    def evoluer(self, population):
        nouvelle_population = Population()
        elite = self.selectionner(population)

        if self.elitisme:
            for individu in elite:
                nouvelle_population.ajouter(individu)

        while len(nouvelle_population.trajets) < self.taille_population:
            parent1, parent2 = random.sample(elite, 2)
            enfant = self.croiser(parent1, parent2)

            if random.random() < self.mut_proba:
                self.muter(enfant)

            nouvelle_population.ajouter(enfant)

        return nouvelle_population

    def executer(self, afficher=True):
        population = Population()
        population.initialiser(self.taille_population, self.liste_villes)
        meilleur_global = population.meilleur()

        for generation in range(self.nb_generations):
            population = self.evoluer(population)
            meilleur_courant = population.meilleur()

            if meilleur_courant.longueur < meilleur_global.longueur:
                meilleur_global = meilleur_courant

            if afficher:
                self.gui.afficher(meilleur_global, meilleur_courant)

        self.gui.window.mainloop()


In [ ]:
# croiser
parent1 = Trajet(Ville.generer_villes(5))
parent2 = Trajet(Ville.generer_villes(5))
algorithme = PVC_Genetique([])
enfant = algorithme.croiser(parent1, parent2)
print("Parent 1 :", parent1)
print("Parent 2 :", parent2)
print("Enfant   :", enfant)

In [ ]:
# muter
trajet = Trajet(Ville.generer_villes(5))
print("Trajet avant mutation :", trajet)
algorithme = PVC_Genetique([])
algorithme.muter(trajet)
print("Trajet après mutation :", trajet)

In [ ]:
# selectionner
population = Population()
population.initialiser(5, Ville.generer_villes(5))
algorithme = PVC_Genetique([])
elite = algorithme.selectionner(population)
print("Population initiale :\n", population)
print("Élite sélectionnée  :\n", elite)

In [ ]:
# evoluer
population = Population()
population.initialiser(5, Ville.generer_villes(5))
algorithme = PVC_Genetique([])
nouvelle_population = algorithme.evoluer(population)
print("Population initiale     :\n", population)
print("Nouvelle population     :\n", nouvelle_population)

In [ ]:
villes = Ville.generer_villes(nb_villes=20)
algorithme = PVC_Genetique(villes, taille_population=40, nb_generations=100)
algorithme.executer(afficher=True)